In [ ]:
# Initializing the project and the keys
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

# Update this line to specify your project id
project_id = "5f00e1ce-d082-4a2b-b08f-889658e932b7"

fablib = fablib_manager(project_id=project_id)

# Import Fablib
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()                     
#fablib.show_config()
import json
import traceback

fablib.verify_and_configure()
!chmod 0600 fabric_config/fabric_bastion_key
!chmod 0600 fabric_config/slice_key

In [ ]:
# Reinitializing the slice or creating a new slice
slice_name = 'My_P4Kube_slice' 
try:
    # Reinitializes the existing slice
    slice = fablib.get_slice(slice_name)
    slice.list_nodes()
    print('Done retrieving the existing slice')
except:
    print(f'Creating a new slice {slice_name}')
    slice = fablib.new_slice(name=slice_name)
    vm_names = ['p4switch', 'master', 'client'] + [f'node{i}' for i in range(1, 11)]
    for vm_name in vm_names:
        node = slice.add_node(name=vm_name, site='RUTG', image='default_ubuntu_20')
        node.set_capacities(cores=2, ram=6, disk=10)
        node.add_component(model='NIC_Basic', name='if1')
    
    # Add additional NICs to p4switch (if2 to if12)
    p4_node = slice.get_node('p4switch')
    for i in range(2, 13):
        p4_node.add_component(model='NIC_Basic', name=f'if{i}')
    
    # Add L2 bridges (ms, cs, n1s-n6s)
    bridge_names = ['ms', 'cs'] + [f'n{i}s' for i in range(1, 11)]
    bridges = {}
    for br in bridge_names:
        bridges[br] = slice.add_l2network(name=br)
    
    # Connect master-if1 and p4switch-if1 to 'ms'
    bridges['ms'].add_interface(slice.get_node('p4switch').get_interface('p4switch-if1-p1'))
    bridges['ms'].add_interface(slice.get_node('master').get_interface('master-if1-p1'))
    
    # Connect client-if1 and p4switch-if2 to 'cs'
    bridges['cs'].add_interface(p4_node.get_interface('p4switch-if2-p1'))
    bridges['cs'].add_interface(slice.get_node('client').get_interface('client-if1-p1'))
    
    # Connect nodes to p4switch
    for i in range(1, 11):
        node = slice.get_node(f'node{i}')
        bridges[f'n{i}s'].add_interface(node.get_interface(f'node{i}-if1-p1'))
        bridges[f'n{i}s'].add_interface(p4_node.get_interface(f'p4switch-if{i+2}-p1'))

    # Submit the slice
    slice.submit()
    print(f"Slice '{slice_name}' created and submitted.")

In [ ]:
print('Changing the mac addresses of all nodes, master and client to match with the controller program')
mac_map = {
    'master':  '5E:00:00:00:00:01',
    'node1':   '5E:00:00:00:00:02',
    'node2':   '5E:00:00:00:00:03',
    'node3':   '5E:00:00:00:00:04',
    'node4':   '5E:00:00:00:00:05',
    'node5':   '5E:00:00:00:00:06',
    'node6':   '5E:00:00:00:00:07',
    'node7':   '5E:00:00:00:00:08',
    'node8':   '5E:00:00:00:00:09',
    'node9':   '5E:00:00:00:00:10',
    'node10':  '5E:00:00:00:00:11',
    'client':  '5E:00:00:00:00:12'
}

# Configuring the routing on the nodes
for node_name in ['master'] + [f'node{i}' for i in range(1, 11)] + ['client']:
    node = slice.get_node(node_name)
    mac = mac_map[node_name]
    if node_name == 'master':
        index = 10
    elif node_name == 'client':
        index = 21
    else:
        index = 10+int(node_name[4:])

    ip = f'{index}.0.0.2/24'
    gw = f'{index}.0.0.1'

    iface = node.get_interface(f"{node_name}-if1-p1").get_os_interface()
    config_script = f"""
        sudo apt-get update && sudo apt-get install -y net-tools;
        sudo ifconfig {iface} down;
        sudo ip link set dev {iface} address {mac};
        sudo ifconfig {iface} {ip};
        sudo ifconfig {iface} up;
        sudo ip route add default via {gw};
    """
    node.execute(config_script)

print('Done, move to the next step')

In [ ]:
print('Configuring IP addresses on the switch, to enable connectivity between hops')
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

p4_node = slice.get_node('p4switch')

switch_ip_map = {
    'ms': '10.0.0.1',
    'cs': '21.0.0.1',
    'n1s': '11.0.0.1',
    'n2s': '12.0.0.1',
    'n3s': '13.0.0.1',
    'n4s': '14.0.0.1',
    'n5s': '15.0.0.1',
    'n6s': '16.0.0.1',
    'n7s': '17.0.0.1',
    'n8s': '18.0.0.1',
    'n9s': '19.0.0.1',
    'n10s': '20.0.0.1'
}

# Add IPs to p4switch's interfaces according to bridge mapping
for bridge, ip in switch_ip_map.items():
    iface = p4_node.get_interface(network_name=bridge)
    subnet = f'{ip[:-1]+"0"}/24'
    iface.ip_addr_add(addr=ip, subnet=IPv4Network(subnet))

# Boot script for switch: installs tools and brings up all interfaces
switch_script = """
    sudo apt-get update && sudo apt-get install -y net-tools;
    for iface in $(ls /sys/class/net | grep enp); do
        sudo ifconfig $iface up
    done;
"""
slice.get_node('p4switch').execute(switch_script)
print('Done, move to the next step')

In [ ]:
print('Installing P4 software (p4c, bmv2) on the P4Switch node.')
p4_node = slice.get_node('p4switch')
switch_script = """
    git clone https://github.com/jafingerhut/p4-guide;
    sudo bash p4-guide/bin/install-p4dev-v5.sh; 
"""
slice.get_node('p4switch').execute(switch_script)
print('Done, move to the next step')

In [ ]:
print('Cloning P4Kube on p4switch node')
p4project = f"""
    git clone https://github.com/gareging/P4Kube.git;
"""
slice.get_node('p4switch').execute(p4project)
print('Done, move to the next step')

In [ ]:
print('Compile P4Kube P4 program')
commands = """
    pip3 install protobuf==3.20.*;
    pip3 install googleapis-common-protos;
    cd P4Kube;
    mkdir build;
    mkdir logs;
    mkdir pcaps;
    p4c-bm2-ss --p4v 16 --p4runtime-files build/p4kube.p4.p4info.txt -o build/p4kube.json p4kube.p4;
"""
slice.get_node('p4switch').execute(commands)
print('Done, move to the next step')

In [ ]:
print('Generating the command to run the switch emulator')
p4switch = slice.get_node('p4switch')

# Port mapping: 1=master, 2=node1, ..., 11=node10, 12=client
bridge_names = ['ms'] + [f'n{i}s' for i in range(1, 11)] + ['cs']
port_map = {}

for port, bridge in enumerate(bridge_names, start=1):
    iface = p4switch.get_interface(network_name=bridge)
    linux_iface = iface.get_os_interface()
    port_map[port] = linux_iface

# Build the interface string
interface_args = ' '.join([f'--interface {port}@{iface}' for port, iface in port_map.items()])
command = f"cd ~/P4Kube; sudo simple_switch_grpc {interface_args} build/p4kube.json"

print('ssh to the switch through a terminal')
print('Run this command in tmux to keep it in background')
print()
print(command)
print()
print("Find the switch ssh command below:", slice.get_node(name="p4switch"))

In [ ]:
print('Initializing the routing table of the P4Kube router')
commands = """
    cd P4Kube;
    python3 mycontroller.py;
"""
slice.get_node('p4switch').execute(commands)
print('All done! Now you can test connectivity between different nodes (e.g., ping client (21.0.0.2) from node1)')

In [ ]:
print('Installing docker, Kubernetes on master and all nodes')
for node_name in ['master'] + [f'node{i}' for i in range(1, 11)]:
    node = slice.get_node(node_name)
    script = """
        # Docker install
        sudo apt update;
        sudo apt upgrade -y;
        sudo apt install docker.io -y;
        sudo systemctl enable docker;
        sudo systemctl start docker;

        # Kubernetes install
        cd /etc/apt && sudo mkdir -p keyrings && cd ~;
        echo "deb [signed-by=/etc/apt/keyrings/kubernetes-apt-keyring.gpg] https://pkgs.k8s.io/core:/stable:/v1.28/deb/ /" | sudo tee /etc/apt/sources.list.d/kubernetes.list;
        curl -fsSL https://pkgs.k8s.io/core:/stable:/v1.28/deb/Release.key | sudo gpg --dearmor -o /etc/apt/keyrings/kubernetes-apt-keyring.gpg;
        sudo apt-get update;
        sudo apt-get install -y kubelet kubeadm kubectl;
        sudo apt-mark hold kubeadm kubelet kubectl;
    """
    node.execute(script)
print('All done, move to the next step')

In [ ]:
print('Setting up the cluster')
node = slice.get_node('master')
script = 'sudo kubeadm init --pod-network-cidr=10.244.0.0/16'
node.execute(script)
print('Update the token in the next cell based on the output above')

In [ ]:
JOIN_COMMAND = 'sudo ' + '''kubeadm join ADD_TOKEN_AND_REMOVE_BACKSLASH '''
print(JOIN_COMMAND)

In [ ]:
print('Joining all worker nodes to the cluster')
for node_name in [f'node{i}' for i in range(1, 11)]:
    node = slice.get_node(node_name)
    node.execute(JOIN_COMMAND)
print('All done, move to the next step')

In [ ]:
print('Configuring networking for Kubernetes via Calico CNI')
node = slice.get_node('master')
script = ''' 
    sudo mkdir -p $HOME/.kube;
    sudo cp -i /etc/kubernetes/admin.conf $HOME/.kube/config;
    sudo chown $(id -u):$(id -g) $HOME/.kube/config;
    kubectl apply -f https://docs.projectcalico.org/manifests/calico.yaml;
    '''
node.execute(script)
print('Done on the master node')

''' Metric help fix '''
for node_name in [f'node{i}' for i in range(1, 11)]:
    node = slice.get_node(node_name)
    ip = f'{10+int(node_name[4:])}.0.0.2'

    script = f'''
        echo "KUBELET_EXTRA_ARGS=--node-ip={ip}" | sudo tee /etc/default/kubelet;
        sudo systemctl daemon-reexec;
        sudo systemctl daemon-reload;
        sudo systemctl restart kubelet;
    '''
    print(f"Configuring kubelet on {node_name}")
    node.execute(script)
print('All done, move to the next step')

In [ ]:
print('Cloning P4Kube repo on the master node')
node = slice.get_node('master')
script = f'git clone https://github.com/gareging/P4Kube.git;'
node.execute(script)
print('All done, move to the next step')

In [ ]:
# Setup the metrics and P4Kube plugin
node = slice.get_node('master')
script = '''
    wget -c https://go.dev/dl/go1.22.0.linux-amd64.tar.gz;
    sudo tar -C /usr/local/ -xzf go1.22.0.linux-amd64.tar.gz;
    export PATH=$PATH:/usr/local/go/bin;
    cd ~/P4Kube/
    kubectl apply -f metrics-server.yaml;
    kubectl patch deployment metrics-server -n kube-system --type 'json' -p '[{"op": "add", "path": "/spec/template/spec/containers/0/args/-", "value": "--kubelet-insecure-tls"}]';
    cd ~/P4Kube/kubectl-plugin;
    go get k8s.io/client-go@latest;
    go get k8s.io/metrics/pkg/client/clientset/versioned;
    go get k8s.io/apimachinery/pkg/util/diff@v0.34.0;
    cp main.go_LOAD_AWARE main.go;
    echo 'export PATH="/home/ubuntu/P4Kube/kubectl-plugin/:$PATH"' >> ~/.bashrc;
    go build;
'''
node.execute(script)

print('All done')

In [ ]:
print("Find the master ssh command below:", slice.get_node(name="master"))
print('ssh into master, open tmux and run the command shown below')
print()
command='''cd ~/P4Kube/kubectl-plugin; export PATH=$PATH:/usr/local/go/bin; echo 'export PATH="/home/ubuntu/P4Kube/kubectl-plugin/:$PATH"' >> ~/.bashrc; go build; kubectl p4-loadbalancer;'''
print(command)

In [ ]:
# Setup the deployment and service (optional)
print('Installing nginx deployment with 10 replicas')
node = slice.get_node('master')
script = '''     
    cd P4Kube/;
    kubectl create -f nginx-deployment.yaml;
    kubectl create -f nginx-service.yaml;
    kubectl patch svc ngnix-service -p '{"spec":{"externalTrafficPolicy":"Local"}}';
'''
node.execute(script)
print('All done, move to the next step')

In [ ]:
print('On the P4Switch node, simple_switch_CLI will show 10 for the register replica_count (1 deployment)')
print('''To check, run 'register_read replica_count' in simple_switch_CLI''')
print("Find the switch ssh command below:", slice.get_node(name="p4switch"))

In [ ]:
'''
This code will automatically replace the default html with a string "nodeX" to observe load balancing 
'''
for node_name in [f'node{i}' for i in range(1, 11)]:
    node = slice.get_node(node_name)
    script = '''
        podid=$(sudo crictl pods | awk '$0 ~ /nginx/ {print $1}');
        dockerid=$(sudo crictl ps | grep $podid | awk '{print $1}');
    '''
    script += f'''echo {node_name} | sudo crictl exec -i "$dockerid" sh -c 'cat > /usr/share/nginx/html/index.html';'''
    node.execute(script)
    print('Done on node', node_name)

In [ ]:
print('Login to client and run curl 10.0.0.2 to verify load balancing')
print("Find the client ssh command below:", slice.get_node(name="client"))

In [ ]:
print('Downscale the first deployment to 7 nodes')
node = slice.get_node('master')
script = 'kubectl scale --replicas=7 deployment nginx'
node.execute(script)
print('Check replica_count register on p4switch')

In [ ]:
print('Install second nginx deployment with 3 replicas')
node = slice.get_node('master')
script = '''     
    cd P4Kube/;
    kubectl create -f nginx-deployment2.yaml;
    kubectl create -f nginx-service2.yaml;
    kubectl patch svc ngnix-service2 -p '{"spec":{"externalTrafficPolicy":"Local"}}';
'''
node.execute(script)
print('On the P4Switch node, replica_count register will show 7, 3 (1st and 2nd deployment)')
print('''To check, run 'register_read replica_count' in simple_switch_CLI''')
print(''' To test from the client, run curl 10.0.0.3 ''')

In [ ]:
print('Install AIOQUIC server as the third deployment')
node = slice.get_node('master')
script = '''     
    cd P4Kube/;
    kubectl create -f quic.yaml;
    kubectl patch svc aioquic-http3-service -p '{"spec":{"externalTrafficPolicy":"Local"}}';
'''
node.execute(script)
print('All done')

In [ ]:
# Testing QUIC (part 1)
print('Install QUIC dependencies on the client')
node = slice.get_node('client')
script = '''     
    sudo apt install libssl-dev python3.9 python3.9-venv python3.9-dev -y;
    sudo apt install python3-pip -y;
    pip3 install aioquic wsproto;
'''
node.execute(script)
print('All done')

In [ ]:
# Testing QUIC (part 2)
print('Clone AIOQUIC repo')
node = slice.get_node('client')
script = '''     
    git clone https://github.com/aiortc/aioquic.git;
'''
node.execute(script)
print('All done')

In [ ]:
# Testing QUIC (part 3)
print('Run QUIC request')
node = slice.get_node('client')
script = '''     
    cd aioquic/examples;
    python3 http3_client.py --insecure https://10.0.0.4:4433/;
'''
node.execute(script)